In [2]:
import numpy as np
import pandas as pd
import random as python_random
import shutil
import os
import datetime
import skimage
import cv2 as cv
from cv2 import THRESH_OTSU
import mediapipe as mp
from datetime import datetime
from skimage import io
from PIL import Image
from multiprocessing import cpu_count
from joblib import Parallel, delayed
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error
from skimage.transform import rescale, resize, downscale_local_mean
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
import skimage
from skimage import color
from skimage.transform import rescale, resize

# for reproducibility purposes
SEED = 123
tf.random.set_seed(SEED)
classes=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

# load tensorboard extension
%reload_ext tensorboard
# specify the log directory where the tensorboard logs will be written
%tensorboard --logdir logs

C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
c:\programdata\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\programdata\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\programdata\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\programda

In [40]:
# takes hand detection output to add bounding rectangles
def find_hands(current, width, height):
    rect_list = []
    hand_classifier = []
    center = (0,0)
    # iterates through hand tracking data per hand
    if current.multi_hand_landmarks:
        for index, info in enumerate(current.multi_handedness):
            which = info.classification[0].label
            hand_classifier.append(which)
        for index, hand in enumerate(current.multi_hand_landmarks):
            # values from 0 to 1 converted into values corresponding to video size
            hand_list = []
            for individual in hand.landmark:
                hand_list.append((int(individual.x * width), int(individual.y * height), int(individual.z * width)))
            # developing bounding box coordinates
            x_values = np.array(hand_list)[:, 0]
            y_values = np.array(hand_list)[:, 1]
            x_min = int(np.min(x_values) - 10)
            y_min = int(np.min(y_values) - 10)
            x_max = int(np.max(x_values) + 10)
            y_max = int(np.max(y_values) + 10)
            center = (x_min + (x_max-x_min)//2, y_min + (y_max-y_min)//2)
            rect_list.append(((x_min, y_min), (x_max, y_max), (0, 255, 0), hand_classifier[index]))
    return rect_list, center
def capture():
    video = cv.VideoCapture(0, cv.CAP_DSHOW) #captureDevice = camera
    running, original = video.read()
    h, w, _ = original.shape    
    # machine learning algorithm (using mediapipe, via google)
    # init_hands = mp.solutions.hands
    # hands = init_hands.Hands()
    hands = mp.solutions.hands.Hands()
    savedFrames = []
    # continuous looping
    while True:
        running, original = video.read()
        if not running:
            break
        # hand processing
        track_curr = hands.process(original)
        # calculating bounding rectangles
        current_rects, center = find_hands(track_curr, w, h)

        if current_rects:
          if len(current_rects) > 1:
            print('Please use only one hand')
          else:
            current_rect = current_rects[0]
            left = current_rect[0][0]
            right = current_rect[1][0]
            top = current_rect[0][1]
            bottom = current_rect[1][1]
            hand_width = right - left
            hand_height = bottom-top
            square_top_left = (center[0]-75,center[1]-75)
            square_bottom_right = (center[0]+75,center[1]+75)

            red = (0,0,255)
            green = (0,255,0)
            if hand_width <= 150 and hand_height <= 150:
              if top > 0 and left > 0 and bottom > 0 and right > 0:
                # cv.rectangle(original, current_rect[0], current_rect[1], green, 1)
                cv.rectangle(original,square_top_left ,square_bottom_right , green,1)
                crop = original[center[1]-75:center[1]+75,center[0]-75:center[0]+75]   
                savedFrames.append(crop)
              
                # cv.circle(original, center, 10, 2, 2)
            else:
              cv.rectangle(original, current_rect[0], current_rect[1], red, 2)
              cv.rectangle(original,square_top_left ,square_bottom_right , red,2)
              print("You're too close! Back up!")
        cv.imshow("Video Feed", original)
        k = cv.waitKey(1) & 0xff
        if k == 27:
            break
    cv.destroyAllWindows()
    video.release()
    return savedFrames
frames = capture()

for i, frame in enumerate(frames):
  frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
  cv.imwrite(f'./custom_dataset/t/{i}.jpg', frame)